In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import langdetect
from time import sleep
from random import randint
import json

In [3]:
f = open('beers_to_scrape.json')
dict_beers = json.load(f)

In [ ]:
page = requests.get("https://www.ratebeer.com/beer/sierra-nevada-pale-ale-bottle-can/365/")

soup = bs(page.content, "html.parser")

beer_table = soup.find_all("dic", {"class": "LinesEllipsis  "})

In [4]:
from selenium import webdriver

DRIVER_PATH = 'chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://www.ratebeer.com/beer/sierra-nevada-pale-ale-bottle-can/365/')
driver.implicitly_wait(7)

#getting the button by class name
button = driver.find_element_by_id("onetrust-reject-all-handler")
button.click()

{'types': {'wheat_beer': ['https://www.ratebeer.com/beer/sierra-nevada-pale-ale-bottle-can/365/', 'https://www.ratebeer.com/beer/stone-ipa-india-pale-ale/422/', 'https://www.ratebeer.com/beer/arrogant-bastard-ale/1315/', 'https://www.ratebeer.com/beer/rogue-dead-guy-ale/589/', 'https://www.ratebeer.com/beer/sierra-nevada-celebration-ale/370/', 'https://www.ratebeer.com/beer/bell-s-two-hearted-ale/1502/', 'https://www.ratebeer.com/beer/anchor-liberty-ale/47/', 'https://www.ratebeer.com/beer/lagunitas-india-pale-ale-ipa/6887/', 'https://www.ratebeer.com/beer/oskar-blues-dale-s-pale-ale/11576/', 'https://www.ratebeer.com/beer/founders-dirty-bastard-scotch-ale/11498/', 'https://www.ratebeer.com/beer/arrogant-double-bastard-ale/11243/', 'https://www.ratebeer.com/beer/dogfish-head-indian-brown-ale/3076/', 'https://www.ratebeer.com/beer/brooklyn-east-ipa-india-pale-ale/530/', 'https://www.ratebeer.com/beer/anchor-old-foghorn-ale/4728/', 'https://www.ratebeer.com/beer/samuel-adams-summer-ale/1